# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

2015 - 2019

### Additioinal sources
This might be useful in matching station codes to airports:
1. http://dss.ucar.edu/datasets/ds353.4/inventories/station-list.html
2. https://www.world-airport-codes.com/

In [3]:
# imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
sqlContext = SQLContext(sc)


In [4]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data"))

path,name,size
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2015.parquet/,2015.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2016.parquet/,2016.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2017.parquet/,2017.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2018.parquet/,2018.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2019.parquet/,2019.parquet/,0


In [5]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201*.parquet")
#display(airlines.sample(False, 0.00001))

In [6]:
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: date (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: boolean (nullable = true)
-- DIVERTED: boolean (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)

In [7]:
f'{airlines.count():,}'

Out[4]: '31,746,841'

In [8]:
display(airlines.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31746841,31274521,31269545,31269545,31269545,31269545,31746841,31260424,31260429,31244917,31244917,31746841,31244919,31176201,31176201,31176201,31176201,31746841,31746677,31178799,31178799,31746841,31746841,31746841,5799114,5799114,5799114,5799114,5799114
mean,2017.1512498204152,2.5174877084620797,6.552106365480585,15.749554640727876,3.9346285509162944,null,12668.724409461716,1266875.803290192,31729.315288031336,null,null,null,26.35374732245013,null,54.91906164774001,12668.666651116562,1266870.0274082704,31729.2951808339,null,null,null,26.354102948384693,null,54.919218135750896,1330.0884999550035,1334.2122192375064,9.855285614165476,12.909587811399238,0.18207943223989986,0.036036885090588944,null,16.830789563186986,1356.9563268309594,1464.4766360877195,7.5604571777227,1488.9034405659447,1468.8957719173475,4.615475952313754,12.966188215170925,0.18601095752494026,-0.20968071125792395,null,143.2167191860742,138.22906985609035,113.8502422431345,1.0,823.2170183483768,3.7652922065537164,19.98459350859459,3.2259498606166392,15.44036813209742,0.08916793151505557,25.364284785572416
stddev,1.4316532810210285,1.1053295681781927,3.3994302561415286,8.774238088354531,1.9917635387471784,null,1526.7397787182167,152673.70669029246,1289.458802620071,null,null,null,16.539517798596837,null,26.577828324534654,1526.7212131574868,152671.85014169724,1289.4192061531862,null,null,null,16.539679261968384,null,26.578079669930972,489.86848319644025,503.2922887741843,43.50520293704072,42.44165318434855,0.3859099860819423,2.161932356946248,null,9.488981863443776,504.9367808166726,531.98737292978,5.929979448174991,516.8048646426242,536.3586689058151,45.59418015238943,42.1408858475887,0.389115517632249,2.2975645036344488,null,74.73117735923346,74.33716296557805,72.2402490397357,0.0,607.6826683052024,2.392350188769286,59.30797970625765,26.81202538233581,34.73908233877255,2.914798174339818,48.60358147038268
min,2015,1,1,1,1,9E,10135,1013503,30070,ABE,"Aberdeen, SD",AK,1,Alabama,1,10135,1013503,30070,ABE,"Aberdeen, SD",AK,1,Alabama,1,1,1,-234.0,0.0,0.0,-2,0001-0559,0.0,1,1,0.0,1,1,-238.0,0.0,0.0,-2,0001-0559,-99.0,14.0,4.0,1.0,21.0,1,0.0,0.0,0.0,0.0,0.0
max,2019,4,12,31,7,YX,16869,1686901,36133,YUM,"Yuma, AZ",WY,78,Wyoming,93,16869,1686901,36133,YUM,"Yuma, AZ",WY,78,Wyoming,93,2359,2400,2755.0,2755.0,1.0,12,2300-2359,227.0,2400,2400,414.0,2400,2400,2695.0,2695.0,1.0,12,2300-2359,948.0,1604.0,1557.0,1.0,5095.0,11,2695.0,2692.0,1848.0,1078.0,2454.0


In [9]:
airlines.where('MONTH == "MONTH"').count()

Out[6]: 0

In [10]:
for year in range(2015, 2020):
  print(year , airlines.select('MONTH').where(f'YEAR == {year}').distinct().collect())

2015 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2016 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2017 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2018 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2019 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]

In [11]:
def nullDataFrame(df):
  null_feature_list = []
  count = df.count()
  for column in df.columns:
    nulls = df.filter(df[column].isNull()).count()
    nulls_perct = np.round((nulls/count)*100, 2)
    null_feature_list.append([column, nulls, nulls_perct])
  nullCounts_df = pd.DataFrame(np.array(null_feature_list), columns=['Feature_Name', 'Null_Counts', 'Percentage_Null_Counts'])
  return nullCounts_df

In [12]:
type(airlines)

Out[13]: pyspark.sql.dataframe.DataFrame

In [13]:
nullCounts_df = nullDataFrame(airlines)
nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,YEAR,0,0.0
1,QUARTER,0,0.0
2,MONTH,0,0.0
3,DAY_OF_MONTH,0,0.0
4,DAY_OF_WEEK,0,0.0
5,FL_DATE,0,0.0
6,OP_UNIQUE_CARRIER,0,0.0
7,ORIGIN_AIRPORT_ID,0,0.0
8,ORIGIN_AIRPORT_SEQ_ID,0,0.0
9,ORIGIN_CITY_MARKET_ID,0,0.0


In [14]:
# 81.73% of records have null/NaN values for [CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY ]
airlines_filtered = airlines.filter(airlines['CARRIER_DELAY'].isNotNull())
f'{airlines_filtered.count():,}'

Out[53]: '5,799,114'

In [15]:
nullCounts_df2 = nullDataFrame(airlines_filtered)
nullCounts_df2

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,YEAR,0,0.0
1,QUARTER,0,0.0
2,MONTH,0,0.0
3,DAY_OF_MONTH,0,0.0
4,DAY_OF_WEEK,0,0.0
5,FL_DATE,0,0.0
6,OP_UNIQUE_CARRIER,0,0.0
7,ORIGIN_AIRPORT_ID,0,0.0
8,ORIGIN_AIRPORT_SEQ_ID,0,0.0
9,ORIGIN_CITY_MARKET_ID,0,0.0


In [16]:
Delay_List = ['ARR_DELAY', 'DEP_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
airlines_filtered[Delay_List].describe().show()

+-------+-----------------+-----------------+-----------------+------------------+-----------------+-------------------+-------------------+
summary| ARR_DELAY| DEP_DELAY| CARRIER_DELAY| WEATHER_DELAY| NAS_DELAY| SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+-------+-----------------+-----------------+-----------------+------------------+-----------------+-------------------+-------------------+
 count| 5799114| 5798779| 5799114| 5799114| 5799114| 5799114| 5799114|
 mean|64.10436421839611|59.83873691340884|19.98459350859459|3.2259498606166392|15.44036813209742|0.08916793151505557| 25.364284785572416|
 stddev|79.32413150747844|82.29469933640682|59.30797970625765| 26.81202538233581|34.73908233877254| 2.9147981743398184| 48.60358147038269|
 min| 15.0| -54.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 max| 2695.0| 2710.0| 2695.0| 2692.0| 1848.0| 1078.0| 2454.0|
+-------+-----------------+-----------------+-----------------+------------------+-----------------+-------------------+-------------------+

In [17]:
sample_airlines_df = airlines_filtered.select(Delay_List).sample(False, 0.001, 2020)
pandas_df = sample_airlines_df.toPandas()

In [18]:
pandas_df.corr()

,ARR_DELAY,DEP_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
ARR_DELAY,1.000000,0.974018,0.574177,0.380086,0.276370,-0.012132,0.521595
DEP_DELAY,0.974018,1.000000,0.576771,0.370367,0.180565,-0.009576,0.548901
CARRIER_DELAY,0.574177,0.576771,1.000000,-0.014378,-0.090110,-0.016235,-0.088504
WEATHER_DELAY,0.380086,0.370367,-0.014378,1.000000,-0.027145,-0.005187,-0.016429
NAS_DELAY,0.276370,0.180565,-0.090110,-0.027145,1.000000,-0.012038,-0.128852
SECURITY_DELAY,-0.012132,-0.009576,-0.016235,-0.005187,-0.012038,1.000000,-0.013939
LATE_AIRCRAFT_DELAY,0.521595,0.548901,-0.088504,-0.016429,-0.128852,-0.013939,1.000000


In [19]:
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
pandas_df.hist(ax=ax, bins=30, figsize=(3,15),)
#ax.set_yscale('log')
plt.yscale('log')
display(plt.show())

# Weather
https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

In [21]:
dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data")

Out[109]: [FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2015a.parquet/', name='2015a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2016a.parquet/', name='2016a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2017a.parquet/', name='2017a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2018a.parquet/', name='2018a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2019a.parquet/', name='2019a.parquet/', size=0)]

In [22]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType
schema = StructType([StructField('STATION', StringType(), True), 
                      StructField('DATE', StringType(), True),
                      StructField('SOURCE', StringType(), True),
                      StructField('LATITUDE', StringType(), True),
                      StructField('LONGITUDE', StringType(), True),
                      StructField('ELEVATION', StringType(), True),
                      StructField('NAME', StringType(), True),
                      StructField('REPORT_TYPE', StringType(), True),
                      StructField('CALL_SIGN', StringType(), True),
                      StructField('QUALITY_CONTROL', StringType(), True),
                      StructField('WND', StringType(), True),
                      StructField('CIG', StringType(), True),
                      StructField('VIS', StringType(), True),
                      StructField('TMP', StringType(), True),
                      StructField('DEW', StringType(), True),
                      StructField('SLP', StringType(), True),
                      StructField('AA1', StringType(), True),
                      StructField('AA2', StringType(), True),
                      StructField('AJ1', StringType(), True),
                      StructField('AY1', StringType(), True),
                      StructField('AY2', StringType(), True),
                      StructField('GA1', StringType(), True),
                      StructField('GA2', StringType(), True),
                      StructField('GA3', StringType(), True),
                      StructField('GE1', StringType(), True),
                      StructField('GF1', StringType(), True),
                      StructField('IA1', StringType(), True),
                      StructField('KA1', StringType(), True),
                      StructField('KA2', StringType(), True),
                      StructField('MA1', StringType(), True),
                      StructField('MD1', StringType(), True),
                      StructField('MW1', StringType(), True),
                      StructField('OC1', StringType(), True),
                      StructField('OD1', StringType(), True),
                      StructField('SA1', StringType(), True),
                      StructField('UA1', StringType(), True),
                      StructField('REM', StringType(), True),
                      StructField('EQD', StringType(), True)
                    ])



In [23]:
weather = spark.read.option("header", "true")\
                      .schema(schema)\
                      .parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/201*a.parquet")
weather.count()


Out[111]: 626994336

In [24]:
display(weather.where('DATE =="DATE"'))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AA1,AA2,AJ1,AY1,AY2,GA1,GA2,GA3,GE1,GF1,IA1,KA1,KA2,MA1,MD1,MW1,OC1,OD1,SA1,UA1,REM,EQD


In [25]:
#display(weather.describe())

In [26]:
nullCounts_weather_df = nullDataFrame(weather)
nullCounts_weather_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,STATION,0,0.0
1,DATE,0,0.0
2,SOURCE,0,0.0
3,LATITUDE,0,0.0
4,LONGITUDE,0,0.0
5,ELEVATION,0,0.0
6,NAME,4715523,0.75
7,REPORT_TYPE,0,0.0
8,CALL_SIGN,0,0.0
9,QUALITY_CONTROL,0,0.0


In [27]:
#display(weather.sample(False, 0.0000001))

In [28]:
weather[["DATE"]].describe().show()

+-------+-------------------+
summary| DATE|
+-------+-------------------+
 count| 626994336|
 mean| null|
 stddev| null|
 min|2015-01-01T00:00:00|
 max|2019-12-31T23:59:00|
+-------+-------------------+

In [29]:
weather_df = weather.withColumn("DATE_IN_DATEFORMAT",weather['DATE'].cast(DateType()))
weather_df.select('DATE_IN_DATEFORMAT','DATE').show(10,False)

+------------------+-------------------+
DATE_IN_DATEFORMAT|DATE |
+------------------+-------------------+
2019-12-08 |2019-12-08T23:55:00|
2019-12-09 |2019-12-09T00:15:00|
2019-12-09 |2019-12-09T00:35:00|
2019-12-09 |2019-12-09T00:55:00|
2019-12-09 |2019-12-09T01:15:00|
2019-12-09 |2019-12-09T01:35:00|
2019-12-09 |2019-12-09T01:55:00|
2019-12-09 |2019-12-09T02:15:00|
2019-12-09 |2019-12-09T02:35:00|
2019-12-09 |2019-12-09T02:55:00|
+------------------+-------------------+
only showing top 10 rows

In [30]:
#display(weather_df.sample(False, 0.0000001))

In [31]:
# Join Airlines data and Weather data by DATE and AIRPORT

# Stations

In [33]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")

In [34]:
#display(stations)

In [35]:
from pyspark.sql import functions as f
stations.where(f.col('name').contains('JAN MAYEN NOR NAVY'))

Out[138]: DataFrame[usaf: string, wban: string, name: string, country: string, state: string, call: string, lat: string, lon: string, elev: string, begin: string, end: string]

In [36]:
stations.select('name').distinct().count()

Out[139]: 26497

In [37]:
display(stations.select('name').distinct())

name
HATTFJELLDAL-KRUTA
COLLAFIRTH HILL
WINDY HEAD
ST ATHAN
FINTHEN (USA-AF) &
HANAU AAF
WSCHOWA
WROCLAW/STRACHOWICE
VIGNA DI VALLE
SAMOS ISLAND


In [38]:
weather.select('NAME').distinct().count()

Out[20]: 13214

In [39]:
#display(weather.select('name').distinct())